In [1]:
from llm import LLM, LLM_Stream
from db import DB, Corpus, Doc, Line, Form, Predict, PredictRaw, Lemma, LemmaRaw
from datetime import datetime
from sqlmodel import SQLModel, Field, Relationship, UniqueConstraint, create_engine, Session, select, delete


In [2]:
llm = LLM()
# llm.complete()
# llm.debug()

In [3]:
llm.lemmatize(llm.db.get_test_line().line)

[{'word_form': 'Apollonius',
  'lemma': 'Apollonius',
  'translation_en': 'Apollonius',
  'translation_ru': 'Аполлоний',
  'morph_analysis': '',
  'syntax_analysis': 'NOUN, SUBJECT'},
 {'word_form': 'cwæð',
  'lemma': 'cweþan',
  'translation_en': 'said',
  'translation_ru': 'сказал',
  'morph_analysis': 'VERB, PAST, 3SG',
  'syntax_analysis': 'VERB, VERB'}]

In [4]:
# llm_stream = LLM_Stream()
# llm_stream.stream_complete("Напиши короткое стихотворение")
# # llm_stream.debug()

In [5]:
# line: Line = db.tbl(Line, 1).one()
# line.forms

In [6]:
db = llm.db
s = db.s
line = llm.db.get_test_line()
raw = llm.response.raw
raw["created_at"]

'2025-01-11T13:31:06.999206Z'

In [26]:
def prediction_save(promt):
  model = llm.llm
  json = llm.json
  response = llm.response
  raw = llm.response.raw
  prediction = Predict(
    line_id = line.id, # line = line,
    test = True,
    model = model.model,
    temperature = model.temperature, 
    done = raw["done"], 
    at = datetime.fromisoformat(raw["created_at"]),
    **{k:raw[k] for k in ["load_duration", "prompt_eval_duration", "eval_duration"]},
    **{k:raw["usage"][k] for k in ["prompt_tokens", "completion_tokens"]},
    no_eq = 0,
    raw=PredictRaw(promt=promt, content=response.text), # type: ignore
  )
  for form in line.forms:
    d = json[form.num]
    if form.form != d["word_form"]:
      print(f"{form} != {d}")
      return 
    lemma = d["lemma"]
    prediction.lemmas.append(Lemma(form=form, lemma=lemma, eq=(form.lemma == lemma),
      raw=LemmaRaw(raw=d, en=d["translation_en"], ru=d["translation_ru"], morph=d["morph_analysis"], syntax=d["syntax_analysis"])
    ))
  s.add(prediction)
  s.commit()
  db.stat()

prediction_save('bla')
db.df(PredictRaw)

1: corpus
5: doc
2541: line
28300: form
3: predict
3: predictraw
6: lemma
6: lemmaraw


,id,promt,content,forms,lemmas,tool_calls
0,1,bla,"[\n {\n ""word_form"": ""Apollonius""...",None,None,{}
1,2,bla,"[\n {\n ""word_form"": ""Apollonius""...",None,None,{}
2,3,bla,"[\n {\n ""word_form"": ""Apollonius""...",None,None,{}


In [27]:
# llm.response.model_dump()

In [28]:
db.del_test()
db.stat()

1: corpus
5: doc
2541: line
28300: form
0: predict
0: predictraw
0: lemma
0: lemmaraw


In [9]:
# s.exec(delete(PredictRaw))
# s.exec(delete(Lemma))
# s.exec(delete(LemmaRaw))
# s.commit()
# db.stat()